In [264]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: ANYWIDGET_HMR=1


In [4]:
import chromospyce
import numpy as np
import pyarrow as pa

In [266]:
# Loading from local arrow file
with pa.memory_map('sample_data/test.arrow', 'r') as source:
    table_local_file = pa.ipc.open_stream(source).read_all()
    
# arr = loaded_arrays[0]
# print(f"{arr[0]} .. {arr[-1]}")
print(table_local_file)

pyarrow.Table
x: double
y: double
z: double
----
x: [[1.101124,-0.919687,-0.974526,-0.881394,-0.455539,...,-6.536989,-8.005743,-8.434383,-6.642313,-5.508548]]
y: [[0.547027,1.275041,1.144577,-0.01193,-2.518781,...,7.12371,6.43679,6.780828,7.377457,8.773718]]
z: [[-2.299305,-3.201912,-4.491504,-4.496625,-2.50066,...,-0.646818,0.229201,1.666676,2.143932,3.108167]]


In [267]:
# Make an Arrow Table from the x, y, z coordinate arrays
# table = pa.Table.from_arrays(decons_data, names=["x", "y", "z"])

# Convert the Table to bytes
output_stream = pa.BufferOutputStream()
with pa.ipc.RecordBatchStreamWriter(output_stream, table_local_file.schema) as writer:
    writer.write_table(table_local_file)

N = 200
numbers = list(range(1, N+1))

vc = {
    "color": {
        "values": numbers,
        "min": 1,
        "max": 100,
        "colorScale": "Spectral"
    }, 
    "scale": 0.02, "links": True, "mark": "sphere"
}

# Get the bytes from the output stream
table_bytes = output_stream.getvalue().to_pybytes()
# Show the structure via chromospyce widget
chromospyce.Widget(structure=table_bytes, viewconfig=vc)

Widget(structure=b'\xff\xff\xff\xff\xc8\x00\x00\x00\x10\x00\x00\x00\x00\x00\n\x00\x0c\x00\x06\x00\x05\x00\x08\…

## Loading a local Arrow file

## Passing a numpy array

In [5]:
import random

def make_random_3D_chromatin_structure(n):
    position = np.array([0.0, 0.0, 0.0])
    positions = [position.copy()]
    x_arr = []
    y_arr = []
    z_arr = []
    for _ in range(n):
        step = np.random.choice([-1.0, 0.0, 1.0], size=3)  # Randomly choose to move left, right, up, down, forward, or backward
        position += step
        positions.append(position.copy())
    return positions

def deconstruct_into_column_arrays(nparr):
    x_arr = []
    y_arr = []
    z_arr = []
    for row in nparr:
        x_arr.append(row[0])
        y_arr.append(row[1])
        z_arr.append(row[2])
        
    return [x_arr, y_arr, z_arr]

In [6]:
## Use case 1: Passing raw numpy array
test_data = make_random_3D_chromatin_structure(n=100)
data = np.array(test_data)

decons_data = deconstruct_into_column_arrays(data)
# decons_data

In [7]:
# Make an Arrow Table from the x, y, z coordinate arrays
table = pa.Table.from_arrays(decons_data, names=["x", "y", "z"])

# Convert the Table to bytes
output_stream = pa.BufferOutputStream()
with pa.ipc.RecordBatchStreamWriter(output_stream, table.schema) as writer:
    writer.write_table(table)

# Get the bytes from the output stream
table_bytes = output_stream.getvalue().to_pybytes()
# table_bytes

In [8]:
# Show the structure via chromospyce widget
vc = {"color": "#6600ff", "scale": 0.02, "links": True, "mark": "sphere"}

chromospyce.Widget(structure=table_bytes, viewconfig=vc)

Widget(structure=b'\xff\xff\xff\xff\xc8\x00\x00\x00\x10\x00\x00\x00\x00\x00\n\x00\x0c\x00\x06\x00\x05\x00\x08\…

## Parsing an XYZ string

In [216]:
# ## Use case 2: Passing XYZ string
test_string = """6
sample = 0
CA	1.101124	0.547027	-2.299305
CA	-0.919687	1.275041	-3.201912
CA	-0.974526	1.144577	-4.491504
CA	-0.881394	-0.011930	-4.496625
CA	-0.455539	-2.518781	-2.500660
CA	-0.626368	-2.086400	-3.427676
"""

def parseXYZ(content):
    x_array = []
    y_array = []
    z_array = []
    delimiter='\t'
    lines = content.splitlines()
    # lines
    lineNumber = 0
    firstLine = lines[0]
    lineNum = int(firstLine)
    for line in lines:
        if lineNumber < 2:
            lineNumber = lineNumber + 1
            continue
        if lineNumber > lineNum:
            break
        lineNumber = lineNumber + 1
        tokens = line.split(delimiter)
        if len(tokens) < 4:
            continue
        
        x = float(tokens[1])
        y = float(tokens[2])
        z = float(tokens[3])
        x_array.append(x)
        y_array.append(y)
        z_array.append(z)

    return [x_array, y_array, z_array]

# x_array
res = parseXYZ(test_string)
x_array = res[0]
y_array = res[1]
z_array = res[2]
# Make an Arrow Table from the x, y, z coordinate arrays
table2 = pa.Table.from_arrays([x_array, y_array, z_array], names=["x", "y", "z"])

# Convert the Table to bytes
output_stream2 = pa.BufferOutputStream()
with pa.ipc.RecordBatchStreamWriter(output_stream2, table2.schema) as writer:
    writer.write_table(table2)

# Get the bytes from the output stream
table2_bytes = output_stream2.getvalue().to_pybytes()

# chromospyce.Widget(structure_string=test_string, delimiter="\t")
chromospyce.Widget(structure=table2_bytes)

Widget(structure=b"\xff\xff\xff\xff\xc8\x00\x00\x00\x10\x00\x00\x00\x00\x00\n\x00\x0c\x00\x06\x00\x05\x00\x08\…

In [218]:
# ## Use case 3: Passing XYZ file path
# import pathlib
# test_path = pathlib.Path("./sample_data/test.xyz")

with open('./sample_data/test.xyz', 'r') as file:
    content = file.read()
    # print(content)
    res = parseXYZ(content)
    x_array = res[0]
    y_array = res[1]
    z_array = res[2]
    # Make an Arrow Table from the x, y, z coordinate arrays
    table3 = pa.Table.from_arrays([x_array, y_array, z_array], names=["x", "y", "z"])
    
    # Convert the Table to bytes
    output_stream3 = pa.BufferOutputStream()
    with pa.ipc.RecordBatchStreamWriter(output_stream3, table3.schema) as writer:
        writer.write_table(table3)
    
    # Get the bytes from the output stream
    table3_bytes = output_stream3.getvalue().to_pybytes()
    
    # chromospyce.Widget(structure_string=test_string, delimiter="\t")
    # chromospyce.Widget(structure=table3_bytes)

# supplying an array of values for the color/size

N = 200
numbers = list(range(1, N+1))

vc = {
    "color": {
        "values": numbers,
        "min": 1,
        "max": 100,
        "colorScale": "Spectral"
    }, 
    "scale": 0.02, "links": True, "mark": "sphere"
}

chromospyce.Widget(structure=table3_bytes, viewconfig=vc)

Widget(structure=b'\xff\xff\xff\xff\xc8\x00\x00\x00\x10\x00\x00\x00\x00\x00\n\x00\x0c\x00\x06\x00\x05\x00\x08\…